In [5]:
import tensorflow as tf
import keras
import sys
sys.path.append("..")

from detection.model import MaskRCNN
from detection.config import Config
import os
import csv

print(tf.__version__)
print(keras.__version__)

2.12.0
2.12.0


In [20]:
# Root directory of the models
ROOT_DIR = os.path.abspath(os.path.join("..", "detection", "models"))

# Path to trained weights file
COCO_MODEL_PATH = os.path.join(ROOT_DIR, "model.h5")

# Directory to save logs and model checkpoints
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

# Model path (hardcoded for simplicity)
model_path = os.path.join(ROOT_DIR, "logs", "mask_rcnn_taco_0100.h5")

# Define your parameters
dataset_path = os.path.join('..', 'data')
round_number = 0
class_map_path = os.path.join('..', 'detection', 'dataset_configurations', 'map.csv')

In [16]:
# Class map
class_map = {}
map_to_one_class = {}
with open(class_map_path) as csvfile:
    reader = csv.reader(csvfile)
    class_map = {row[0]: row[1] for row in reader}
    map_to_one_class = {c: 'Litter' for c in class_map}
print(class_map)

{'Aerosol': 'Can', 'Aluminium foil': 'Other', 'Battery': 'Other', 'Aluminium blister pack': 'Other', 'Carded blister pack': 'Other', 'Clear plastic bottle': 'Bottle', 'Glass bottle': 'Bottle', 'Other plastic bottle': 'Bottle', 'Plastic bottle cap': 'Bottle cap', 'Metal bottle cap': 'Bottle cap', 'Broken glass': 'Other', 'Drink can': 'Can', 'Food Can': 'Can', 'Corrugated carton': 'Other', 'Drink carton': 'Other', 'Egg carton': 'Other', 'Meal carton': 'Other', 'Other carton': 'Other', 'Paper cup': 'Cup', 'Disposable plastic cup': 'Cup', 'Foam cup': 'Cup', 'Glass cup': 'Cup', 'Other plastic cup': 'Cup', 'Food waste': 'Other', 'Plastic lid': 'Lid', 'Metal lid': 'Lid', 'Magazine paper': 'Other', 'Tissues': 'Other', 'Wrapping paper': 'Other', 'Normal paper': 'Other', 'Paper bag': 'Other', 'Plastified paper bag': 'Other', 'Pizza box': 'Other', 'Garbage bag': 'Plastic bag + wrapper', 'Single-use carrier bag': 'Plastic bag + wrapper', 'Polypropylene bag': 'Plastic bag + wrapper', 'Produce bag':

In [18]:
from detection.dataset import Taco

# Create a new Taco dataset
dataset_test = Taco()

# Load the data
taco = dataset_test.load_taco(dataset_path, round_number, "test", class_map=class_map, return_taco=True)

# Prepare the dataset
dataset_test.prepare()

# Get the number of classes
nr_classes = dataset_test.num_classes
print(nr_classes)

creating index...
index created!
Number of images used: 150
11


In [21]:
# Prepare the config
class TacoTestConfig(Config):
            NAME = "taco"
            GPU_COUNT = 1
            IMAGES_PER_GPU = 1
            DETECTION_MIN_CONFIDENCE = 0
            NUM_CLASSES = nr_classes
            USE_OBJECT_ZOOM = False
config = TacoTestConfig()

# Create a new Mask R-CNN model in inference mode
model = MaskRCNN(mode="inference", config=config, model_dir=DEFAULT_LOGS_DIR)

# Load weights
model.load_weights(model_path, model_path, by_name=True)

# Convert the model to TFLite
converter = tf.lite.TFLiteConverter.from_keras_model(model.keras_model)

# Set quantization parameters (so it doesn't break)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
                                       tf.lite.OpsSet.SELECT_TF_OPS]
tflite_model = converter.convert()

# Save the TFLite model
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\LENOVO\AppData\Local\Temp\tmprrwvqu87\assets


INFO:tensorflow:Assets written to: C:\Users\LENOVO\AppData\Local\Temp\tmprrwvqu87\assets
